In [2]:
import os
import librosa   #thư viện xủ lý tín hiệu âm thanh
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #để đọc file wav 
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [3]:
import matplotlib.pyplot as plt
import librosa.display

from numpy import load
from numpy import save
from numpy import asarray
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'noise']
all_wave = load('all_wave.npy')
# 
all_label = load('all_label.npy')
all_feature=[]
for word in all_wave:
#     feature_vector_1 = librosa.feature.chroma_stft(y=word, sr=16000)
#     feature_vector_2 = librosa.feature.melspectrogram(y=word, sr=16000)[:40]
#     feature_vector = np.concatenate((feature_vector_1*10,feature_vector_2))
#     all_feature.append(feature_vector)
    all_feature.append(word)

In [5]:
#data preprocessing
# labels=os.listdir(train_audio_path)
from sklearn.preprocessing import LabelEncoder
all_feature = np.array(all_feature)
feature_vector = all_feature[0]
print(all_feature.shape)
feature_size = feature_vector.shape

le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)
from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))
# all_feature = np.array(all_feature).reshape(-1,feature_size[0],feature_size[1],1)
all_feature = np.array(all_feature).reshape(-1,feature_size[0],1)
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_feature),np.array(y),stratify=y,test_size = 0.3,random_state=555,shuffle=True)

(4965, 6000)


In [6]:
#thiết kế mô hình deep learning
from keras.models import Model, Sequential
from keras.layers import *
from keras.activations import *
from keras.optimizers import Adam,RMSprop,SGD
model1 = Sequential()
model1.add(Conv1D(8, kernel_size=13, strides=1, padding='valid',
                  input_shape=(feature_size[0],1)))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling1D(pool_size=3))
model1.add(Dropout(0.3))

model1.add(Conv1D(16, kernel_size=11, strides=1, padding='valid'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling1D(pool_size=3))
model1.add(Dropout(0.3))

model1.add(Conv1D(32, kernel_size=9, strides=1, padding='valid'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling1D(pool_size=3))
model1.add(Dropout(0.3))

model1.add(Conv1D(64, kernel_size=7, strides=1, padding='valid'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling1D(pool_size=3))
model1.add(Dropout(0.3))

# model1.add(Conv1D(128, kernel_size=5, strides=1, padding='valid'))
# model1.add(LeakyReLU(alpha=0.1))
# model1.add(MaxPooling1D(pool_size=3))
# model1.add(Dropout(0.3))
           
model1.add(Flatten())

model1.add(Dense(64))
model1.add(Dropout(0.1))

model1.add(Dense(11))
# model1.add(Dropout(0.1))

model1.add(Activation('softmax'))
# adam = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model1.summary()

# learning rate schedule
def lr_schedule(epoch):
    if epoch < 10:
        return 0.005
    elif epoch < 40:
        return 0.002
    elif epoch < 80:
        return 0.001
    else:
        return 0.0005
callbacks = [LearningRateScheduler(schedule=lr_schedule, verbose=1),
#              TensorBoard("./", histogram_freq=0, write_graph=True, write_images=False)
#              ,
#              ModelCheckpoint(
#                  os.path.join("./", "cp_{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5"),
#                  monitor='val_loss', verbose=1, save_best_only=True)
            ]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 5988, 8)           112       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5988, 8)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1996, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1996, 8)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 16)          1424      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1986, 16)          0         
______________________________________

In [7]:
#luyện mô hình 
classifier = model1.fit(x_tr,
                    y_tr,validation_data=( x_val, y_val),
                    callbacks=callbacks,
                    validation_steps = 5,
                    steps_per_epoch = 20,
                    epochs=25,
                    batch_size=None)


Train on 3475 samples, validate on 1490 samples
Epoch 1/25

Epoch 00001: LearningRateScheduler setting learning rate to 0.005.
20/20 [==============================] - 12s 591ms/step - loss: 2.0213 - accuracy: 0.4123 - val_loss: 0.3620 - val_accuracy: 2.2450
Epoch 2/25

Epoch 00002: LearningRateScheduler setting learning rate to 0.005.
20/20 [==============================] - 9s 460ms/step - loss: 1.6365 - accuracy: 0.4828 - val_loss: 0.3079 - val_accuracy: 2.4799
Epoch 3/25

Epoch 00003: LearningRateScheduler setting learning rate to 0.005.
20/20 [==============================] - 9s 461ms/step - loss: 1.5478 - accuracy: 0.5231 - val_loss: 0.2764 - val_accuracy: 2.9228
Epoch 4/25

Epoch 00004: LearningRateScheduler setting learning rate to 0.005.
20/20 [==============================] - 9s 462ms/step - loss: 1.2573 - accuracy: 0.6085 - val_loss: 0.2254 - val_accuracy: 3.2416
Epoch 5/25

Epoch 00005: LearningRateScheduler setting learning rate to 0.005.
20/20 [========================

In [8]:
# lưu kiến trúc của model đã luyện vào file json
model_json = model1.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# lưu các trọng số của model
model1.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [5]:
print("* init")
import os
import IPython.display as ipd
# from scipy.io import wavfile #để đọc file wav 
# import warnings
import tensorflow as tf
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import socket
import pyaudio
import wave
import struct
import numpy as np
CHUNK = 16000 # read each 1000 miliseconds
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 20
WORD = 6000
WORD_HALF = int(WORD/2)
WORD_QUAD = int(WORD/2)
# circular_buf_size = 50
p = pyaudio.PyAudio()
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
model.summary()
# open the file for reading.
wf = wave.open("test.wav", 'rb')
# open stream based on the wave object which has been input.
stream = p.open(format =
                p.get_format_from_width(wf.getsampwidth()),
                channels = wf.getnchannels(),
                rate = wf.getframerate(),
                input = True)
# stream = p.open(format=FORMAT,
#                 channels=CHANNELS,
#                 rate=RATE,
#                 input=True,
#                 frames_per_buffer=CHUNK)
# stream = p.open("test.wav",format=FORMAT,
#                 channels=CHANNELS,)
frames = []
features = []
waves = []
words = []
circular_index = 0
os.system( 'cls' )
print("* recording")
inputDataNew = np.ndarray(shape=(CHUNK), dtype=float)
inputDataOld = np.ndarray(shape=(WORD_QUAD), dtype=float)
last_word_time = 0
i=0
while 1:
    data = stream.read(CHUNK) 
    #convert to float array
    for j in range(0, int(len(data)/4)):
        value = struct.unpack('f', data[j*4:j*4+4])
        inputDataNew[j] = value[0]
    #megre end of old and new
    inputData = np.concatenate((inputDataOld,inputDataNew))
    inputDataOld = inputDataNew[CHUNK-WORD_QUAD:].copy()
    i=i+1
    if i<2:
        continue
    else:
        mean = np.mean(inputData)
        energy = abs(inputData)
        value = 0
        max_val = 0
        max_idn = 0
        for idn in range(0,len(energy)):
            value += (energy[idn]-value)/300
            if (idn>WORD_HALF) & (idn<(len(inputData)-WORD_HALF )):
                if max_val<value:
                    max_val = value
                    max_idn = idn
        if max_val>(mean*6):
            global_time = i*CHUNK+max_idn-WORD_QUAD
            if (global_time-last_word_time)< WORD:
                continue
            last_word_time = global_time
            data = inputData[max_idn-WORD_HALF:max_idn+WORD_HALF]
#             feature_vector_1 = librosa.feature.chroma_stft(y=data, sr=16000)
#             feature_vector_2 = librosa.feature.melspectrogram(y=data, sr=16000)[:40]
            feature_vector = data #np.concatenate((feature_vector_1*10,feature_vector_2))
            modelInput = np.array(feature_vector).reshape(1,feature_vector.shape[0],1)
            output = model.predict(modelInput)
            indexMax = np.argmax(output[0], axis=0) 
            confident = output[0][indexMax]
            if (confident>0.8):
#             os.system( 'cls' )
                print ("ID:",len(waves)," prediction:",indexMax,"confident:",output[0][indexMax]," time:",global_time/RATE)
                waves.append(inputData)
                words.append(data)
                features.append(feature_vector)
                byte_message = (indexMax)
                opened_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                opened_socket.sendto(byte_message, ("127.0.0.1", 5005))
#             plt.figure(figsize=(10, 4))
#             librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
#             plt.colorbar()
#             plt.title('Chromagram')
#             plt.tight_layout()
#             plt.show()
print("* done recording")    

* init
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 5988, 8)           112       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5988, 8)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1996, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1996, 8)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 16)          1424      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1986, 16)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 662, 16)   

KeyboardInterrupt: 